In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("../data/night/data_night.csv", encoding='cp949')

In [10]:
df.columns

Index(['Unnamed: 0', '국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약',
       '대표청구항', '청구항 수', '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일',
       '출원인', '출원인 국적', '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '발명자/고안자',
       '우선권 번호', '우선권 국가', '우선권 주장일', '국제 공개번호', '국제 공개일', 'Original CPC Main',
       'Original CPC All', 'Original IPC Main', 'Original IPC All',
       'Original US Class Main[US]', 'Original US Class All[US]',
       'WIPS ON key', '유효특허 여부'],
      dtype='object')

In [11]:
df.shape

(1236, 34)

### convert column's name to english & drop columns

In [12]:
df = df.rename(index=str, 
               columns={'등록번호': 'grant_id','출원번호':'application_id', '공개번호/공표/재공표 ':'published_grant_id',
                        '발명의 명칭': "title", '대표청구항':'claim', '청구항 수':'n_claim', "발명자/고안자":"inventor",
                        '우선권 번호':'priority_id', '요약': 'abstract', 'Unnamed: 0' : 'index', '등록일':'grant_date',
                        '출원일':'application_date', '공개일':'published_date', '출원인':'applicant', 
                        '출원인 수':'n_applicant',
                        '출원인 대표명화 코드':'applicant_code', '국제 공개번호':'international_published_id', 
                        'Original IPC Main':'ipc_main', 'Original IPC All':'ipc_all',
                        'Original US Class Main[US]':'usclass_main', 'Original US Class All[US]':'usclass_all',
                        '유효특허 여부':'valid_patent', '문헌종류 코드':'patent_kind'})

In [13]:
df = df.drop(['특허/실용 구분', 'DB종류', '국가코드', '출원인 대표명화 영문명', '국제 공개일', '우선권 주장일',
             'Original CPC Main', 'Original CPC All', 'WIPS ON key', '출원인 국적', '우선권 국가'], axis=1)

In [14]:
df.shape

(1236, 23)

### merge edgelist to df

#### ipc main - all에 main이 포함

In [15]:
# main_ipc_edge = pd.read_csv("../data/main_ipc_edge.tsv", sep='\t')
# cols = main_ipc_edge.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# main_ipc_edge = main_ipc_edge[cols]
# main_ipc_edge["value"] = 1
# main_ipc_matrix = pd.pivot(values=main_ipc_edge.value, index=main_ipc_edge['patent'], 
#                            columns=main_ipc_edge['ipc']).fillna(0)

# df = df.drop('ipc_main', axis=1)
# df.index = df['index']
# df = df.drop_duplicates()
# df = df.merge(main_ipc_matrix, left_index=True, right_index=True)

#### usclass main - all에 main이 포함

In [16]:
# main_usclass_edge = pd.read_csv("../data/main_usclass_edge.tsv", sep='\t')
# cols = main_usclass_edge.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# main_usclass_edge = main_usclass_edge[cols]
# main_usclass_edge["value"] = 1
# main_usclass_matrix = pd.pivot(values=main_usclass_edge.value, index=main_usclass_edge['patent'], 
#                            columns=main_usclass_edge['us_class']).fillna(0)

# df = df.drop('usclass_main', axis=1)
# df.index = df['index']
# df = df.drop_duplicates()
# df = df.merge(main_usclass_matrix, left_index=True, right_index=True)

#### ipc all

In [17]:
all_ipc_edge = pd.read_csv("../data/night/night_all_ipc_edge.tsv", sep='\t')
cols = all_ipc_edge.columns.tolist()
cols = cols[-1:] + cols[:-1]
all_ipc_edge = all_ipc_edge[cols]
all_ipc_edge["value"] = 1
all_ipc_matrix = pd.pivot_table(all_ipc_edge, index=['patent', 'ipc'])

all_ipc_matrix_reset_index = all_ipc_matrix.reset_index()
all_ipc_matrix_reset_index = pd.pivot(values= all_ipc_matrix_reset_index.value,
                                      index=all_ipc_matrix_reset_index.patent,
                                      columns=all_ipc_matrix_reset_index.ipc).fillna(0)

df = df.drop('ipc_all', axis=1)
df = df.drop('ipc_main', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(all_ipc_matrix_reset_index, left_index=True, right_index=True)
print(all_ipc_matrix_reset_index.shape)

(1236, 199)


#### usclass all

In [18]:
all_usclass_edge = pd.read_csv("../data/night/night_all_usclass_edge.tsv", sep='\t')
cols = all_usclass_edge.columns.tolist()
cols = cols[-1:] + cols[:-1]
all_usclass_edge = all_usclass_edge[cols]
all_usclass_edge["value"] = 1
all_usclass_matrix = pd.pivot_table(all_usclass_edge, index=['patent', 'us_class'])

all_usclass_matrix_reset_index = all_usclass_matrix.reset_index()
all_usclass_matrix_reset_index = pd.pivot(values= all_usclass_matrix_reset_index.value,
                                          index=all_usclass_matrix_reset_index.patent,
                                          columns=all_usclass_matrix_reset_index.us_class).fillna(0)

df = df.drop('usclass_all', axis=1)
df = df.drop('usclass_main', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(all_usclass_matrix_reset_index, left_index=True, right_index=True)
print(all_usclass_matrix_reset_index.shape)

(1236, 2099)


#### inventor

In [19]:
inventor_edge = pd.read_csv("../data/night/night_inventor_edge.tsv", sep='\t')
cols = inventor_edge.columns.tolist()
cols = cols[-1:] + cols[:-1]
inventor_edge = inventor_edge[cols]
inventor_edge["value"] = 1
inventor_matrix = pd.pivot_table(inventor_edge, index=['patent_id', 'inventor'])

inventor_matrix_reset_index = inventor_matrix.reset_index()
inventor_matrix_reset_index = pd.pivot(values= inventor_matrix_reset_index.value,
                                       index=inventor_matrix_reset_index.patent_id,
                                       columns=inventor_matrix_reset_index.inventor).fillna(0)

df = df.drop('inventor', axis=1)
df.index = df['index']
df = df.drop_duplicates()
df = df.merge(inventor_matrix_reset_index, left_index=True, right_index=True)
print(inventor_matrix_reset_index.shape)

(1236, 2359)


#### applicant

In [20]:
applicant_edge = pd.read_csv("../data/night/night_applicant_edge.tsv", sep='\t')
cols = applicant_edge.columns.tolist()
cols = cols[-1:] + cols[:-1]
applicant_edge = applicant_edge[cols]
applicant_edge["value"] = 1
applicant_matrix = pd.pivot_table(applicant_edge, index=['patent_id', 'applicant'])

applicant_matrix_reset_index = applicant_matrix.reset_index()
applicant_matrix_reset_index = pd.pivot(values= applicant_matrix_reset_index.value,
                                        index=applicant_matrix_reset_index.patent_id,
                                        columns=applicant_matrix_reset_index.applicant).fillna(0)

df = df.drop('applicant', axis=1)
df = df.drop('applicant_code', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(applicant_matrix_reset_index, left_index=True, right_index=True)
print(applicant_matrix_reset_index.shape)

(1236, 502)


<hr>

### Pivot Table

#### assignee

In [14]:
# assignee_edge = pd.read_csv("../data/assignee_edge.tsv", sep='\t')
# cols = assignee_edge.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# assignee_edge = assignee_edge[cols]
# assignee_edge["value"] = 1
# assignee_matrix = pd.pivot_table(assignee_edge, index=['patent_id', 'assignee_id'])

# assignee_matrix_reset_index = assignee_matrix.reset_index()
# assignee_matrix_reset_index = pd.pivot(values= assignee_matrix_reset_index.value,
#                                        index=assignee_matrix_reset_index.patent_id,
#                                        columns=assignee_matrix_reset_index.assignee_id).fillna(0)

# assignee_matrix_reset_index.shape

(309, 261)

#### appication citation

In [15]:
# application_citation_edge = pd.read_csv("../data/application_citation_edge.tsv", sep='\t')
# cols = application_citation_edge.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# application_citation_edge = application_citation_edge[cols]
# application_citation_edge["value"] = 1
# application_citation_matrix = pd.pivot_table(application_citation_edge, 
#                                              index=['patent_id', 'application_citation_id'])

# application_citation_matrix_reset_index = application_citation_matrix.reset_index()
# application_citation_matrix_reset_index = pd.pivot(values= application_citation_matrix_reset_index.value,
#                                                    index=application_citation_matrix_reset_index.patent_id,
#                                                    columns=application_citation_matrix_reset_index.application_citation_id).fillna(0)

# application_citation_matrix_reset_index.shape

(164, 732)

#### grant citaion

In [16]:
# grant_citation_edge = pd.read_csv("../data/grant_citation_edge.tsv", sep='\t')
# cols = grant_citation_edge.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# grant_citation_edge = grant_citation_edge[cols]
# grant_citation_edge["value"] = 1
# grant_citation_matrix = pd.pivot_table(grant_citation_edge, 
#                                              index=['patent_id', 'grant_citation_id'])

# grant_citation_matrix_reset_index = grant_citation_matrix.reset_index()
# grant_citation_matrix_reset_index = pd.pivot(values= grant_citation_matrix_reset_index.value,
#                                              index=grant_citation_matrix_reset_index.patent_id,
#                                              columns=grant_citation_matrix_reset_index.grant_citation_id).fillna(0)

# grant_citation_matrix_reset_index.shape

(435, 6694)

In [21]:
df.shape

(1236, 5175)

In [22]:
df.to_csv("../data/night/night_dataset.csv", index=False, header=True)